## Quantum Heat Engine Based on Strained Monolayer Graphene
Trying to reproduce the results of the engine proposed in Pena and Munoz (2018).

The primary reference is:<br>
FJ Pena, E Munoz. 2018. _Magneto-strain-driven quantum engine based on a graphene flake_. Physical Review E. [__91__, 052152](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.91.052152). [arXiv:1505.04208](https://arxiv.org/abs/1505.04208).

In [1]:
using Printf
using Unitful, UnitfulRecipes
using Plots
using Roots
using LinearAlgebra
using LaTeXStrings

# upscaled plot
upscale = 2.4
fntsm = Plots.font(pointsize=round(5.5*upscale))
fntlg = Plots.font(pointsize=round(6.0*upscale))
default(titlefont=fntlg, guidefont=fntlg, tickfont=fntsm, legendfont=fntsm, 
        legendtitlefont=fntlg)
default(size=(400*upscale,300*upscale))
default(linewidth=upscale)

# units
const eV, T, K, ° =  u"eV", u"T", u"K", u"°"

# universal constants
const e = 1.602e-19u"C" 
const h = 4.136e-15u"eV*s"
const ħ = h/2π
const k_B = 8.617e-5u"eV/K"
const μ_B = 5.788e-5u"eV/T"
const m_e = 9.109e-31u"kg"

# graphene constants
const a = 2.46e-10u"m"               # lattice constant
λ(j) = ([3.16, 0.381]u"eV")[j+1]     # λ(0), λ(1) parametrize in-plane, dimer hoppings
v(j) = (√3a*λ(j)) / 2ħ               # v(0), v(1) fermi velocities
const m = λ(1) / (2v(0)^2)           # effective mass of quasiparticles
const γ = 0.9μ_B                     # Zeeman coupling
const γ̄  = 1.7μ_B                    # pseudo-Zeeman coupling
const A = (1e3a)^2                   # area of sample

l(B) = sqrt(ħ/(e*abs(B)))            # Landau radius
β(T) = (k_B*T)^-1                    # thermodynamic β

Ω(ξ, B, B_S) = v(0) * sqrt(2*e*abs(B + ξ*B_S)/ħ)
Δ(ξ, B, B_S) = (γ̄*B_S) / (ħ*Ω(ξ, B, B_S))
N(ξ, B, B_S) = floor((abs(B + ξ*B_S) * A) / (h/e))

function E(T, B)
    # From Equation B6
    return (-γ*B*tanh(β(T)*γ*B) 
            + (sum([N(ξ, B, B_S) * (ξ*γ̄*B_S*exp(-ξ*β(T)*γ̄*B_S) 
                                    + (ħ*Ω(ξ, B, B_S)*sqrt(1 + Δ(ξ, B, B_S)^2) 
                                       * exp(-β(T)*ħ*Ω(ξ, B, B_S)*sqrt(1 + Δ(ξ, B, B_S)^2)))) 
                    for ξ in [-1, 1]]) 
               / sum([N(ξ, B, B_S) * (exp(-ξ*β(T)*γ̄*B_S) 
                                      + exp(-β(T)*ħ*Ω(ξ, B, B_S)*sqrt(1 + Δ(ξ, B, B_S)^2))) 
                      for ξ in [-1, 1]])))
end

function S(T, B)
    # From Equation B7
    return (β(T)*E(T, B) 
            + log(2*cosh(β(T)*γ*B)) 
            + log(sum([N(ξ, B, B_S) * (exp(-ξ*β(T)*γ̄*B_S) 
                                       + exp(-β(T)*ħ*Ω(ξ, B, B_S)*sqrt(1 + Δ(ξ, B, B_S)^2)))
                      for ξ in [-1, 1]])))
end

function otto(r)
    B_2 = r^2 * B_1

    # Intermediate temperatures determined numerically. Equation 36.
    T_2 = find_zero(T -> S(T_C, B_1) - S(T, B_2), T_C)
    T_4 = find_zero(T -> S(T_H, B_2) - S(T, B_1), T_H)

    Q_H = E(T_H, B_2) - E(T_2, B_2)   # Equation 33
    Q_C = E(T_C, B_1) - E(T_4, B_1)   # Equation 34
    W = Q_C + Q_H                     # Equations 31 and 32
    η = 1 - abs(Q_C/Q_H)              # Equation 35

    @printf("r_B: %.3f. T_2: %.4f K. T_4: %.4f K. W: %4f meV. η: %4f.\n", 
            r, ustrip(K, T_2), ustrip(K, T_4), ustrip(u"meV", W), η)
end

B_1, B_S = 4.0T, 20.0T
T_C, T_H = 30.0K, 100.0K

(30.0 K, 100.0 K)

In [2]:
r_B = 1.0:0.1:2.5
otto.(r_B);

r_B: 1.000. T_2: 30.0000 K. T_4: 100.0000 K. W: 0.000000 meV. η: 0.000000.
r_B: 1.100. T_2: 30.6171 K. T_4: 96.9626 K. W: 0.016942 meV. η: 0.017621.
r_B: 1.200. T_2: 31.0788 K. T_4: 97.1572 K. W: 0.022550 meV. η: 0.023295.
r_B: 1.300. T_2: 31.5223 K. T_4: 97.6093 K. W: 0.026558 meV. η: 0.027261.
r_B: 1.400. T_2: 31.8953 K. T_4: 98.3576 K. W: 0.028031 meV. η: 0.028626.
r_B: 1.500. T_2: 32.1688 K. T_4: 99.4949 K. W: 0.026098 meV. η: 0.026561.
r_B: 1.600. T_2: 32.3054 K. T_4: 101.1517 K. W: 0.019593 meV. η: 0.019921.
r_B: 1.700. T_2: 32.2551 K. T_4: 103.5217 K. W: 0.006882 meV. η: 0.007014.
r_B: 1.800. T_2: 31.9058 K. T_4: 107.1855 K. W: -0.016395 meV. η: -0.016852.
r_B: 1.900. T_2: 31.2072 K. T_4: 112.2365 K. W: -0.050990 meV. η: -0.053263.
r_B: 2.000. T_2: 29.8421 K. T_4: 120.5931 K. W: -0.109287 meV. η: -0.117878.
r_B: 2.100. T_2: 27.3560 K. T_4: 135.8596 K. W: -0.207454 meV. η: -0.238060.
r_B: 2.200. T_2: 21.7546 K. T_4: 176.2115 K. W: -0.413146 meV. η: -0.553785.
r_B: 2.300. T_2: 22.